In [1]:
import platform, sys, os

# Install WorkingWithFiles
To install WorkingWithFiles go to next link https://github.com/trucomanx/WorkingWithFiles


# Variables

In [2]:
## Seed for the random variables
seed_number=0;

## Model of network
model_type = 'mobilenet_v3';
#model_type = 'efficientnet_b3'
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';

times=1;

categories=['angry','disgusted','fearful','happy','neutral','sad','surprised'];



# Endereços

In [3]:


## Dataset 
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        dataset_base_dir    = 'fer2013/archive/test';
    elif 'microsoft-standard' in platform.uname().release:
        dataset_base_dir    = '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/test';
    else:
        dataset_base_dir    = '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test';
else:
    dataset_base_dir    = 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\test';

print('dataset_base_dir:',dataset_base_dir)

## Output
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        output_base_dir = 'OUTPUTS/cnn_face_emotion/test_custom';
    elif 'microsoft-standard' in platform.uname().release:
        output_base_dir = '/mnt/c/Dados/Fernando/OUTPUTS/cnn_face_emotion/test_custom';
    else:
        output_base_dir = '/mnt/boveda/DOCTORADO2/cnn_face_emotion/test_custom';
else:
    output_base_dir = 'C:\\Dados\\Fernando\\OUTPUTS\\cnn_face_emotion\\test_custom';

print('output_base_dir:',output_base_dir)

dataset_base_dir: /mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test
output_base_dir: /mnt/boveda/DOCTORADO2/cnn_face_emotion/test_custom


# Biblioteca Local

In [4]:
sys.path.append('library');
import Classifier as mylib

# Bibliotecas externas

In [5]:
import PIL
import WorkingWithFiles as rnfunc
from tensorflow.keras.preprocessing.image import load_img

# If command line

In [6]:
#print('cmd entry:', sys.argv)

for n in range(len(sys.argv)):
    if sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    if sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);

print('model_type:',model_type)
print('times:',times)

model_type: mobilenet_v3
times: 1


# Classifier

In [7]:
Clf=mylib.FaceEmotionClassifier(model_type);

Loading model: mobilenet_v3
Model mobilenet_v3 loaded.
Loaded layer with mobilenet_v3


# Create directory

In [15]:
#output_dir = os.path.join(output_base_dir,'holdout');
output_dir = os.path.join(output_base_dir,'delay_'+model_type);

try: 
    os.makedirs(output_dir) 
except: 
    pass

# Time

In [9]:
from pytictoc import TicToc
t = TicToc() 

# Testing people

In [10]:

Ntot=0;
Ltot=0;
Ttot=0;
for id in range(len(categories)):
    category=categories[id];
    
    print('category:',category);
    basedir=os.path.join(dataset_base_dir,category);
    total_list=rnfunc.get_all_files_in_dir_list([basedir],formats_search=['.png']);

    N0=0;
    L0=len(total_list);

    pil_image=[];
    for n in range(L0):
        pil_image.append(load_img(total_list[n]));

    t.tic();
    for m in range(times):
        for n in range(L0):
            res=Clf.is_pil_patient(pil_image[n]);
            N0=N0+(res==id);
    t0=t.tocvalue();

    print(' Acc:',category,':',N0*1.0/(L0*times));
    print('Time:',category,':',t0,'s');

    Ntot=Ntot+N0*1.0/times;
    Ltot=Ltot+L0;
    Ttot=Ttot+t0*1.0/times;

category: angry
 Acc: angry : 0.6106471816283925
Time: angry : 37.963171420000435
category: disgusted
 Acc: disgusted : 0.6216216216216216
Time: disgusted : 4.382489024001188
category: fearful
 Acc: fearful : 0.4418377321603128
Time: fearful : 40.792522636000285
category: happy
 Acc: happy : 0.8776775648252536
Time: happy : 71.25668668599974
category: neutral
 Acc: neutral : 0.6063311688311688
Time: neutral : 48.90729219499917
category: sad
 Acc: sad : 0.5164658634538153
Time: sad : 49.86975493099999
category: surprised
 Acc: surprised : 0.8204819277108434
Time: surprised : 32.56683067499944


# Resultados finais

In [16]:
acc=Ntot*1.0/Ltot;
delayms=Ttot*1000.0/Ltot;

print('Acc:',acc);
print('Delay',delayms,'ms')

Acc: 0.6599749058971142
Delay 39.8353196106232 ms


In [17]:
fpath=os.path.join(output_dir,"times"+str(times)+"_acc_delayms.m")

fid = open(fpath, 'w')
print('delayms={}'.format(delayms),';', file = fid);
print('acc={}'.format(acc),';', file = fid);
fid.close()